In [1]:
from datasets import load_dataset

ds = load_dataset("cq01/mawps-asdiv-a_svamp")

README.md:   0%|          | 0.00/761 [00:00<?, ?B/s]

(…)-00000-of-00001-52e35a24f615aa7b.parquet:   0%|          | 0.00/527k [00:00<?, ?B/s]

(…)-00000-of-00001-1d9ca8ce89c3de29.parquet:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3138 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [2]:
!pip install --upgrade pip

!pip uninstall unsloth -y
!pip uninstall torch torchvision -y
!pip uninstall xformers -y

!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"
!pip install torch==2.4.1 torchvision==0.19.1
!pip install --upgrade xformers==0.0.28.post1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-r5radcn0/unsloth_5cf6380b9718419588f362770ea75e53
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-r5radcn0/unsloth_5cf6380b9718419588f362770ea75e53
  Resolved https://github.com/unslothai/unsloth.git to commit f26d4e739ed507de7a9088da53d10fd02f58d160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━

In [3]:
import os
import torch
import numpy as np
import pandas as pd

In [4]:
for key, value in ds.items():
    print(f"Length of '{key}': {len(value)}")

Length of 'train': 3138
Length of 'validation': 1000


In [5]:
import torch

# Check if a GPU is available
print(torch.cuda.is_available())

True


In [6]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [7]:
def generate_full_question(example):
    question = example["Question"]
    numbers = example["Numbers"]
    for i, num in enumerate(numbers):
        placeholder = f"number{i}"
        # Check if the number is an integer
        if num.is_integer():
            num_str = str(int(num))
        else:
            num_str = str(num)
        question = question.replace(placeholder, num_str)
    example["Full Question"] = question
    return example

# Apply the transformation to the train and validation sets
ds = ds.map(generate_full_question)

# Now you can access the transformed dataset with the new "Full Question" column
print(ds["train"][23]["Full Question"])

Map:   0%|          | 0/3138 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

A construction company ordered 0.16666666666666666 ton of concrete , 0.16666666666666666 ton of bricks , and 0.5 ton of stone . How many tons of material did the company order in all ?


In [8]:
print(ds["train"][0]["Full Question"])
print(ds["train"][1]["Full Question"])
print(ds["train"][2]["Full Question"])

Bryan took a look at his books as well . If Bryan has 56 books in each of his 9 bookshelves , how many books does he have in total ?
For the fifth grade play , the chairs have been put into 27 rows with 16 chairs in each row . How many chairs have been put out for the play ?
There are 41 short trees and 44 tall trees currently in the park . Park workers will plant 57 short trees today . How many short trees will the park have when the workers are finished ?


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

local_model_path = "unsloth/gemma-2-9b"
# local_model_path = "/kaggle/input/gemma-2-9b/other/default/1/models/mathGemma-2-9b"

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    device_map="auto",  
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from tqdm import tqdm

def extract_number_eval(text):
    # Extract the text after 'the answer is'
    # match = re.search(r'the answer is\s*(.*)', text, re.IGNORECASE)
    match = re.search(r'### Response:\s*the answer is\s*(.*)', text, re.IGNORECASE | re.DOTALL)
    if match:
        number_text = match.group(1).strip()
        # print('Number text is:', number_text)
        number_text = re.sub(r'[^0-9\.\-\+eE]', '', number_text)
        # print('After removing non-digit characters, number_text is:', number_text)
        if number_text.count('.') > 1:
            parts = number_text.split('.', 1)
            number_text = parts[0] + '.' + parts[1].replace('.', '')
        try:
            number = float(number_text)
            return number
        except ValueError:
            numbers = re.findall(r"[-+]?\d*\.?\d+", text)
            return float(numbers[-1]) if numbers else None
    else:
        numbers = re.findall(r"[-+]?\d*\.?\d+", text)
        return float(numbers[-1]) if numbers else None
    
    
def extract_number_eval2(text):
    matches = re.findall(r'the answer is (\d+\.?\d*)', text)
    if matches:
        last_answer = float(matches[-1])  # Get the last match
    else:
        last_answer = extract_number_eval(text)
        
    return last_answer

def create_few_shot_examples(num_shots=3):
    """Create few-shot examples with CoT reasoning."""
    examples = [
        {
            "question": "every day swayam spends 4 hours on learning english and 9 hours on learning chinese. how many more hours does he spend on learning chinese than he does on learning english?",
            "reasoning": """Let's think step by step:
1. Hours spent on Chinese = 9 hours
2. Hours spent on English = 4 hours
3. Hours spent more on Chinese than on English = Hours on Chinese - Hours on English = 9 - 4
Therefore, the answer is 5"""
        },
        {
            "question": "A store offers a 20% discount on a $80 shirt. What is the final price?",
            "reasoning": """Let's think step by step:
1. Calculate the discount amount: $80 × 20% = $80 × 0.2 = $16
2. Subtract discount from original price: $80 - $16 = $64
Therefore, the answer is 64"""
        },
        {
            "question": "A recipe needs 2.5 cups of flour for 4 servings. How many cups are needed for 6 servings?",
            "reasoning": """Let's think step by step:
1. Find flour per serving: 2.5 ÷ 4 = 0.625 cups
2. Multiply by new servings: 0.625 × 6
Therefore, the answer is 3.75"""
        }
    ]
    return examples[:num_shots] if num_shots > 0 else []
    

def evaluate_accuracy(model, tokenizer, eval_dataset, num_samples=100, 
                      results_file="gemini_evaluation_results.txt",
                      responses_file="gemini_detailed_responses.txt"):
    """Evaluate model accuracy on a small subset of validation data."""
    model.eval()
    correct = 0
    total = 0

    eval_subset = eval_dataset

    initial_results = []

    evaluation_prompt =  """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You have to solve a math problem. Provide a brief, concise step-by-step solution and then provide the final numerical answer in the format: "Therefore, the answer is [number]"

Here are some examples:

Example-1: 
### Input:
every day swayam spends 4 hours on learning english and 9 hours on learning chinese. how many more hours does he spend on learning chinese than he does on learning english?

### Response:
Let's think step by step:
1. Hours spent on Chinese = 9 hours
2. Hours spent on English = 4 hours
3. Hours spent more on Chinese than on English = Hours on Chinese - Hours on English = 9 - 4
Therefore, the answer is 5


Example-2:
### Input:
a store offers a 20% discount on a $80 shirt. what is the final price?

### Response:
Let's think step by step:
1. Calculate the discount amount: $80 × 20% = $80 × 0.2 = $16
2. Subtract discount from original price: $80 - $16 = $64
Therefore, the answer is 64


Now solve the following math problem after thinking about the problem step by step:

### Input:
{question}

### Response:
Let's think step by step:"""

    # Open the results file for writing
    with open(results_file, "w") as file, open(responses_file, "w") as resp_file:
        file.write("Evaluation Results:\n\n")

        print(f"\nEvaluating accuracy on {num_samples} examples...")
        progress_bar = tqdm(eval_subset, total=num_samples)
        for example in progress_bar:
            try:
                # Prepare input
                prompt = evaluation_prompt.format(question=example["Full Question"])
                inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

                # Generate prediction
                with torch.no_grad():
                    outputs = model.generate(
                        input_ids=inputs["input_ids"],
                        max_new_tokens=512,
                        use_cache=False,
                        do_sample=False,
                    )

                # Decode output
                generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

                # Extract prediction and true answer
                predicted_number = extract_number_eval2(generated_text)
                true_answer = float(example["Answer"])

                # Check if correct
                is_correct = (predicted_number is not None and abs(predicted_number - true_answer) < 1e-6)
                correct += int(is_correct)
                total += 1

                # Store result
                result = {
                    "question": example["Full Question"],
                    "true_answer": true_answer,
                    "predicted": predicted_number,
                    "correct": is_correct
                }
                
                initial_results.append(result)
                
                results_line = (f"Question: {result['question']}\n"
                                f"True Answer: {result['true_answer']}\n"
                                f"Predicted: {result['predicted']}\n"
                                f"Correct: {result['correct']}\n\n")

                # Write result to file
                file.write(results_line)
            
                # Write detailed response
                detailed_response = (f"Question: {result['question']}\n"
                                     f"Complete Response:\n{generated_text}\n"
                                     f"True Answer: {result['true_answer']}\n"
                                     f"Predicted: {result['predicted']}\n"
                                     f"Correct: {result['correct']}\n"
                                     f"{'-'*50}\n\n")
                resp_file.write(detailed_response)
                
                progress_bar.set_postfix(accuracy=f"{(correct / total) * 100:.2f}%")

            except Exception as e:
                error_message = f"Error processing example: {e}\n"
                print(error_message)
                file.write(error_message)
                continue

        accuracy = correct / total if total > 0 else 0
        accuracy_message = f"\nAccuracy on {num_samples} samples: {accuracy:.2%}\n"
        print(accuracy_message)
        file.write(accuracy_message)

    return accuracy, initial_results

In [11]:
torch.cuda.empty_cache()
import gc
gc.collect()

21

In [12]:
#### Random test experiment
import re
import time

example = ds["validation"][51]

evaluation_prompt =  """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You have to solve a math problem. Provide a brief, concise step-by-step solution and then provide the final numerical answer in the format: "Therefore, the answer is [number]"

Here are some examples:

Example-1: 
### Input:
every day swayam spends 4 hours on learning english and 9 hours on learning chinese. how many more hours does he spend on learning chinese than he does on learning english?

### Response:
Let's think step by step:
1. Hours spent on Chinese = 9 hours
2. Hours spent on English = 4 hours
3. Hours spent more on Chinese than on English = Hours on Chinese - Hours on English = 9 - 4
Therefore, the answer is 5


Example-2:
### Input:
a store offers a 20% discount on a $80 shirt. what is the final price?

### Response:
Let's think step by step:
1. Calculate the discount amount: $80 × 20% = $80 × 0.2 = $16
2. Subtract discount from original price: $80 - $16 = $64
Therefore, the answer is 64


Now solve the following math problem after thinking about the problem step by step:

### Input:
{question}

### Response:
Let's think step by step:"""

stime = time.time()

prompt = evaluation_prompt.format(question=example["Full Question"])
# print(prompt)
# print("*****************\n\n\n")

# inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
inputs = tokenizer(
        prompt, 
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=512,  # Match with max_seq_length
).to(model.device)

model.eval()

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        eos_token_id=tokenizer.eos_token_id,  
        max_new_tokens=200,
        use_cache=False,
        do_sample=False,
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)
print('\n\n**********************\n\n')
predicted_number = extract_number_eval(generated_text)
print(predicted_number)

etime = time.time()
print(etime - stime)

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 107

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You have to solve a math problem. Provide a brief, concise step-by-step solution and then provide the final numerical answer in the format: "Therefore, the answer is [number]"

Here are some examples:

Example-1: 
### Input:
every day swayam spends 4 hours on learning english and 9 hours on learning chinese. how many more hours does he spend on learning chinese than he does on learning english?

### Response:
Let's think step by step:
1. Hours spent on Chinese = 9 hours
2. Hours spent on English = 4 hours
3. Hours spent more on Chinese than on English = Hours on Chinese - Hours on English = 9 - 4
Therefore, the answer is 5


Example-2:
### Input:
a store offers a 20% discount on a $80 shirt. what is the final price?

### Response:
Let's think step by step:
1. Calculate the discount amount: $80 × 20% = $80 × 0.2 =

In [13]:
ds["validation"] = ds["validation"].select(list(range(50))) 
print(len(ds["validation"]))


50


In [14]:
print("\nEvaluating model accuracy after fine-tuning...")
initial_accuracy, initial_results = evaluate_accuracy(
    model, tokenizer, ds["validation"], num_samples=len(ds["validation"])
)



Evaluating model accuracy after fine-tuning...

Evaluating accuracy on 50 examples...


100%|██████████| 50/50 [3:14:10<00:00, 233.00s/it, accuracy=96.00%]   


Accuracy on 50 samples: 96.00%

